In [12]:
!curl -L -o file_conduitbugbash_taxi_2019_ORQXQ2K7GIYDCOI_snappy_parquet.tar.gz https://www.dropbox.com/s/aeqbb0fsv4klvsc/file_conduitbugbash_taxi_2019_ORQXQ2K7GIYDCOI_snappy_parquet.tar.gz?dl=0

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1034    0  1034    0     0   1171      0 --:--:-- --:--:-- --:--:--  1171
100  101M  100  101M    0     0  26.2M      0  0:00:03  0:00:03 --:--:-- 41.6M


In [ ]:
!tar -xzvf file_conduitbugbash_taxi_2019_ORQXQ2K7GIYDCOI_snappy_parquet.tar.gz
!mkdir -p /mnt/conduit_parquet_store
!mv file_conduitbugbash_taxi_2019_ORQXQ2K7GIYDCOI_snappy_parquet /mnt/conduit_parquet_store/

In [ ]:
#for HADOOP you have to do
!hadoop fs -mkdir /mnt
!hadoop fs -mkdir /mnt/conduit_parquet_store
!hadoop fs -copyFromLocal -r file_conduitbugbash_taxi_2019_ORQXQ2K7GIYDCOI_snappy_parquet /mnt/conduit_parquet_store/

In [1]:
import os
import sys
import psutil
import time
import cudf
from blazingsql import BlazingContext
from dask.distributed import Client
import dask_cudf


In [3]:
LOCAL_FOLDER="/mnt/conduit_parquet_store/file_conduitbugbash_taxi_2019_ORQXQ2K7GIYDCOI_snappy_parquet"
HDFS_FOLDER = "hdfs://172.31.3.117:9000/mnt/conduit_parquet_store/file_conduitbugbash_taxi_2019_ORQXQ2K7GIYDCOI_snappy_parquet"


In [4]:
client = Client('172.31.3.117:18085')
bc = BlazingContext(dask_client=client, network_interface='ens5',allocator="default",pool=False)

BlazingContext ready


In [5]:
DISTRIBUTED = True
DASK_SCHEDULER_IP = "172.31.3.117"
DASK_SCHEDULER_PORT = "18085"
NETWORK_INTERFACE = "ens5"
HDFS_IP="172.31.3.117"
HDFS_MASTER = "172.31.3.117:9000"
CONDUIT_HDFS="conduithdfs"
bc.hdfs(f"{CONDUIT_HDFS}", host=f"{HDFS_IP}", port=9000, user="bde")

(True,
 '',
 OrderedDict([('type', 'hdfs'),
              ('host', '172.31.3.117'),
              ('port', 9000),
              ('user', 'bde'),
              ('driver', 'libhdfs'),
              ('kerberos_ticket', '')]))

In [6]:
target_folder_name = f"hdfs://{HDFS_MASTER}{LOCAL_FOLDER}"            
print(target_folder_name)
local_df = dask_cudf.read_parquet(target_folder_name)
local_df = bc.unify_partitions(local_df)
bc.create_table("taxi", local_df)

hdfs://172.31.3.117:9000/mnt/conduit_parquet_store/file_conduitbugbash_taxi_2019_ORQXQ2K7GIYDCOI_snappy_parquet


In [8]:
bc.sql("select * from taxi limit 10").compute()

MemoryError: std::bad_alloc: CUDA error at: /conda/conda-bld/librmm_1585667632672/work/include/rmm/mr/device/cuda_memory_resource.hpp66: cudaErrorMemoryAllocation out of memory

In [ ]:
"""
in dask_worker-space folder
-rw-r--r-- 1 root root 7.4G Apr 10 11:56 %28%27from-delayed-f875f4ea22b7530f6e770023bcae298c%27%2C%200%29
drwxr-xr-x 2 root root 4.0K Apr 10 11:59 ./
drwx------ 3 root root 4.0K Apr 10 07:24 ../
-rw-r--r-- 1 root root  12K Apr 10 11:59 collectPartitionsRunQuery-d8e5ec196afd1156e4cf0867151d1e3d
-rw-r--r-- 1 root root 4.5K Apr 10 11:59 getTableScanInfoCaller-6a0dbf4f2e83c247e1c6663adaf774de
-rw-r--r-- 1 root root 7.4G Apr 10 11:54 workerUnifyPartitions-cb8f901d52ec7e41ed476f45ff569142
"""